<a href="https://colab.research.google.com/github/jaalvalcan/GEE_index_sets/blob/main/Q_C_Points_before__random__forest_application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import ee
import geemap
import json
import geopandas as gpd
import  fiona



In [36]:
# Initialize Earth Engine and map
ee.Authenticate()
ee.Initialize(project='ee-jaalvalcan')

In [37]:
# Create map centered on your area
Map = geemap.Map(basemap='SATELLITE')
Map.addLayerControl()

In [39]:
!pip install geopandas fiona


In [46]:
def load_and_fix_control_points(ground_truth_paths):
    """Load control points and fix 3D coordinate issue"""

    all_points = []
    success_count = 0
    total_points_loaded = 0

    print("🚀 LOADING AND FIXING 3D COORDINATES...")

    for class_name, file_path in ground_truth_paths.items():
        try:
            print(f"\n📁 Processing {class_name}...")

            # Load with GeoPandas
            gdf = gpd.read_file(file_path)
            print(f"   Found {len(gdf)} points")

            # Convert each geometry individually
            class_points = []
            for idx, row in gdf.iterrows():
                try:
                    geom = row.geometry

                    if geom.geom_type == 'Point':
                        # Extract coordinates and remove the Z value (elevation)
                        coords_3d = list(geom.coords)[0]  # This gives (lon, lat, elevation)
                        coords_2d = coords_3d[:2]  # Take only (lon, lat)

                        # Convert to EE Geometry with 2D coordinates
                        ee_geom = ee.Geometry.Point(coords_2d)

                        # Create feature with class property
                        feature = ee.Feature(ee_geom, {'class': class_name})
                        class_points.append(feature)

                        if idx < 3:  # Show first 3 conversions
                            print(f"     Converted: {coords_3d} → {coords_2d}")

                    else:
                        print(f"   ⚠️ Skipping non-point geometry at index {idx}: {geom.geom_type}")

                except Exception as point_error:
                    print(f"   ❌ Failed to convert point {idx}: {point_error}")
                    continue

            if class_points:
                all_points.extend(class_points)
                success_count += 1
                total_points_loaded += len(class_points)
                print(f"   ✅ Successfully converted {len(class_points)}/{len(gdf)} points")
            else:
                print(f"   ❌ No points could be converted for {class_name}")

        except Exception as e:
            print(f"   ❌ Error processing {class_name}: {e}")

    # Final summary
    print("\n" + "="*60)
    print("FIXED LOADING SUMMARY:")
    print(f"📊 Classes processed: {len(ground_truth_paths)}")
    print(f"✅ Successfully loaded: {success_count} classes")
    print(f"📍 Total points loaded: {total_points_loaded}")

    if all_points:
        control_points_fc = ee.FeatureCollection(all_points)
        print(f"\n🎉 SUCCESS! All {total_points_loaded} control points are ready!")
        print("✨ Next: Visualize points and extract Sentinel-2 spectral values")
        return control_points_fc
    else:
        print("\n💥 All conversions failed")
        return None

# Try the fixed version
control_points = load_and_fix_control_points(ground_truth_paths)

🚀 LOADING AND FIXING 3D COORDINATES...

📁 Processing Barefield...
   Found 10 points
     Converted: (-83.82641274183324, 14.26920966316919, 0.0) → (-83.82641274183324, 14.26920966316919)
     Converted: (-83.8259238409793, 14.26889827352941, 0.0) → (-83.8259238409793, 14.26889827352941)
     Converted: (-83.82834364152357, 14.27063532432695, 0.0) → (-83.82834364152357, 14.27063532432695)
   ✅ Successfully converted 10/10 points

📁 Processing Forest...
   Found 6 points
     Converted: (-83.83367303577434, 14.26653644522441, 0.0) → (-83.83367303577434, 14.26653644522441)
     Converted: (-83.83328187648311, 14.26574563052269, 0.0) → (-83.83328187648311, 14.26574563052269)
     Converted: (-83.83304658766559, 14.26808747058367, 0.0) → (-83.83304658766559, 14.26808747058367)
   ✅ Successfully converted 6/6 points

📁 Processing Grassland...
   Found 9 points
     Converted: (-83.37022202612876, 14.30736111687069, 0.0) → (-83.37022202612876, 14.30736111687069)
     Converted: (-83.37026244

In [47]:
def verify_fixed_points(control_points):
    """Verify that the points are properly loaded and converted"""

    print("\n🔍 VERIFYING FIXED CONTROL POINTS...")

    # Get basic info
    point_count = control_points.size().getInfo()
    print(f"Total points in collection: {point_count}")

    # Check class distribution
    class_stats = control_points.aggregate_histogram('class').getInfo()
    print("\n📊 Class distribution:")
    for class_name, count in class_stats.items():
        print(f"  {class_name}: {count} points")

    # Check first few points
    print(f"\n📍 Sample points (first 3):")
    sample_points = control_points.limit(3)
    point_info = sample_points.getInfo()

    for i, feature in enumerate(point_info['features']):
        coords = feature['geometry']['coordinates']
        class_name = feature['properties']['class']
        print(f"  {i+1}. {class_name}: {coords}")

    print("✅ Verification complete!")

# Run verification
verify_fixed_points(control_points)


🔍 VERIFYING FIXED CONTROL POINTS...
Total points in collection: 48

📊 Class distribution:
  Barefield: 10 points
  Forest: 6 points
  Grassland: 9 points
  Urban: 13 points
  Water: 10 points

📍 Sample points (first 3):
  1. Barefield: [-83.82641274183324, 14.26920966316919]
  2. Barefield: [-83.8259238409793, 14.26889827352941]
  3. Barefield: [-83.82834364152357, 14.27063532432695]
✅ Verification complete!


In [48]:
def visualize_with_sentinel(Map, control_points, aoi):
    """Visualize control points with Sentinel-2 background"""

    print("\n🗺️ CREATING VISUALIZATION WITH SENTINEL-2...")

    # Get cloud-free Sentinel-2 composite
    def get_sentinel_composite(aoi):
        s2_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                  .filterBounds(aoi)
                  .filterDate('2024-01-01', '2024-12-31')
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)))

        return s2_col.median().clip(aoi)

    # Get Sentinel-2 composite
    s2_composite = get_sentinel_composite(aoi)

    # Add Sentinel-2 as background (True Color)
    vis_params = {
        'min': 0,
        'max': 3000,
        'bands': ['B4', 'B3', 'B2']  # RGB
    }

    Map.addLayer(s2_composite, vis_params, 'Sentinel-2 True Color')

    # Define colors for each class
    class_colors = {
        'Barefield': 'brown',
        'Forest': 'darkgreen',
        'Grassland': 'lightgreen',
        'Urban': 'red',
        'Water': 'blue'
    }

    # Add each class with different color and larger points
    for class_name, color in class_colors.items():
        class_points = control_points.filter(ee.Filter.eq('class', class_name))
        point_count = class_points.size().getInfo()

        if point_count > 0:
            Map.addLayer(
                class_points,
                {
                    'color': color,
                    'pointSize': 5,
                    'pointShape': 'diamond',
                    'width': 2
                },
                f'🏷️ {class_name} ({point_count} points)'
            )
            print(f"✅ Added {class_name} points")

    # Center map on AOI with good zoom level
    Map.centerObject(aoi, 11)

    # Add layer control
    Map.addLayerControl()

    print("\n🎯 MAP READY!")
    print("• Check if points are correctly located on their land cover types")
    print("• Verify Forest points on trees, Water on water bodies, etc.")
    print("• Look for any misclassified points")

    return Map

# Create and display the map
print("Creating visualization map...")
Map = visualize_with_sentinel(Map, control_points, aoi)
Map

Creating visualization map...

🗺️ CREATING VISUALIZATION WITH SENTINEL-2...
✅ Added Barefield points
✅ Added Forest points
✅ Added Grassland points
✅ Added Urban points
✅ Added Water points

🎯 MAP READY!
• Check if points are correctly located on their land cover types
• Verify Forest points on trees, Water on water bodies, etc.
• Look for any misclassified points


Map(center=[14.33676168638322, -83.32443249999972], controls=(WidgetControl(options=['position', 'transparent_…